In [143]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [144]:
# import dataset and load in a dataframe
customer_file = r'..\data\customers_tm1_e.csv'
transaction_file = r'..\data\transactions_tm1_e.csv'
cust_df = pd.read_csv(customer_file)
trans_df = pd.read_csv(transaction_file)

In [145]:
cust_df.head()

,customer_id,dob,state,start_balance,creation_date
0,91,1993-07-01,California,10180.56,2007-01-31
1,92,1985-12-05,New York,4757.68,2007-01-31
2,93,1987-11-19,Minnesota,6796.72,2007-01-31
3,94,1981-03-23,Minnesota,9870.48,2007-01-31
4,95,1970-04-06,California,12500.72,2007-01-31


In [146]:
trans_df.head()

,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal
0,2007-01-31,24137947,91,3034.26,2007-01-31,3034.26,0.00
1,2007-01-31,24137947,91,-5295.18,2007-01-16,0.00,-5295.18
2,2007-02-28,24137947,91,0.00,2007-02-28,0.00,0.00
3,2007-03-31,24137947,91,-0.00,2007-03-30,0.00,-0.00
4,2007-03-31,24137947,91,-0.00,2007-03-11,0.00,-0.00


In [147]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116016 entries, 0 to 116015
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   customer_id    116016 non-null  int64  
 1   dob            116016 non-null  object 
 2   state          116016 non-null  object 
 3   start_balance  116013 non-null  float64
 4   creation_date  116016 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 4.4+ MB


In [148]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4977972 entries, 0 to 4977971
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   date              object 
 1   account_id        int64  
 2   customer_id       int64  
 3   amount            float64
 4   transaction_date  object 
 5   deposit           float64
 6   withdrawal        float64
dtypes: float64(3), int64(2), object(2)
memory usage: 265.9+ MB


In [149]:
is_missing = trans_df.isna()

missing_per_column = is_missing.sum(axis = 0) 
print(missing_per_column)              

date                0
account_id          0
customer_id         0
amount              0
transaction_date    0
deposit             0
withdrawal          0
dtype: int64


##### Replacing the Na in starting balance with 0

In [150]:
# replacing empty start balances with 0, but this should be edited with transaction data if the information is there.
cust_df['start_balance'].fillna(0, inplace=True)

In [151]:
print(f"Customer columns:\n{list(cust_df.columns)}\n")
print(f"Transactions columns:\n{list(trans_df.columns)}")

Customer columns:
['customer_id', 'dob', 'state', 'start_balance', 'creation_date']

Transactions columns:
['date', 'account_id', 'customer_id', 'amount', 'transaction_date', 'deposit', 'withdrawal']


#### Casting to date type

In [152]:
trans_df['date'] = pd.to_datetime(trans_df['date'])
trans_df['transaction_date'] = pd.to_datetime(trans_df['transaction_date'])
cust_df['creation_date'] = pd.to_datetime(cust_df['creation_date'])

##### Checking if amount is equal to the sum of deposit and withdrawal

In [153]:
trans_df['amount'].equals(trans_df["deposit"] + trans_df["withdrawal"])

False

In [154]:
#Since it doesn't match, I am going to copy the table and see if I can find anything interesting about it
ttemp_df = trans_df.copy()

In [155]:
ttemp_df["transaction_sum"] = ttemp_df["withdrawal"] + ttemp_df["deposit"]
ttemp_df["transaction_diff"] = ttemp_df["amount"] - ttemp_df["transaction_sum"]
filter1 = ttemp_df[ttemp_df["transaction_diff"] != 0]

In [156]:
# 18 rows have aounts that don't equal the sum of the withdrawal and deposit
filter1.count()

date                18
account_id          18
customer_id         18
amount              18
transaction_date    18
deposit             18
withdrawal          18
transaction_sum     18
transaction_diff    18
dtype: int64

In [157]:
# The rows with a difference between the amount and transaction sum, 
# Have amounts that are significantly different from the transaction sum
display(filter1[['customer_id', 'transaction_date', 'amount', 'transaction_sum', 'transaction_diff']])

,customer_id,transaction_date,amount,transaction_sum,transaction_diff
556196,9822,2007-08-31,-1.000000e+10,1703.45,-1.000000e+10
556197,9822,2007-08-12,-1.000000e+10,-73.91,-1.000000e+10
556198,9822,2007-08-27,-1.000000e+10,-898.84,-9.999999e+09
556199,9822,2007-08-29,-1.000000e+10,-102.68,-1.000000e+10
556200,9822,2007-08-10,-1.000000e+10,-291.17,-1.000000e+10
556201,9822,2007-08-17,-1.000000e+10,-463.53,-1.000000e+10
556202,9822,2007-08-12,-1.000000e+10,-749.91,-9.999999e+09
1419211,30441,2011-10-31,-1.000000e+10,1577.41,-1.000000e+10
1419212,30441,2011-10-24,-1.000000e+10,-1005.71,-9.999999e+09
1419213,30441,2011-10-04,-1.000000e+10,-685.87,-9.999999e+09


In [158]:
# Option 1 is to adjust amount so that it matches the transaction sum
ttemp_df["amount"] = np.where(ttemp_df["transaction_diff"] != 0, ttemp_df["transaction_sum"], ttemp_df["amount"])

In [159]:
ttemp_df["transaction_diff"] = ttemp_df["amount"] - ttemp_df["transaction_sum"]
filter1 = ttemp_df[ttemp_df["transaction_diff"] != 0]
filter1.count()

date                0
account_id          0
customer_id         0
amount              0
transaction_date    0
deposit             0
withdrawal          0
transaction_sum     0
transaction_diff    0
dtype: int64

In [160]:
# The other option would be be to change the withdrawal and deposits to match amount

Fixing the amount

making it equal the sum of withdrawal and deposit

In [161]:
trans_df["amount"] = np.where(trans_df["amount"] != trans_df["withdrawal"] + trans_df["deposit"], trans_df["withdrawal"] + trans_df["deposit"], trans_df["amount"])

In [162]:
trans_df['amount'].equals(trans_df["deposit"] + trans_df["withdrawal"])


True

#### Adding a rolling count of transactions for each customer

In [163]:
trans_df['transaction_number'] = trans_df.sort_values(['customer_id', 'transaction_date'], ascending=True).groupby(['customer_id', 'account_id']).cumcount()+1


## Looking at cust_df

#### Checking start_balance

In [164]:
filter2 = cust_df[cust_df["start_balance"] <= 0]
filter2.count()

customer_id      5
dob              5
state            5
start_balance    5
creation_date    5
dtype: int64

In [165]:
display(filter2[["customer_id", "start_balance", "creation_date"]])

,customer_id,start_balance,creation_date
21243,21334,0.000000e+00,2010-01-31
37823,37914,0.000000e+00,2011-10-31
42613,42704,-1.000000e+10,2012-04-30
45891,45982,0.000000e+00,2012-08-31
95939,96030,0.000000e+00,2017-10-31


In [166]:
state_dict = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 
              'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 
              'District Of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 
              'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 
              'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 
              'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 
              'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 
              'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 
              'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 
              'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 
              'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 
              'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 
              'Wyoming':'WY'}

for key, value in state_dict.items():
    cust_df["state"].replace(value, key, inplace= True)

In [167]:
cust_df.groupby(["state"])["customer_id"].count()

state
-999                        3
Alabama                  1765
Alaska                    242
Arizona                  2576
Arkansas                 1005
Australia                   3
CALIFORNIA                  3
California              14289
Colorado                 2093
Connecticut              1284
Delaware                  369
District of Columbia      241
Florida                  7592
Georgia                  3808
Hawaii                    534
Idaho                     658
Illinois                 4630
Indiana                  2292
Iowa                     1097
Kansas                   1094
Kentucky                 1591
Louisiana                1660
MASS                        3
Maine                     501
Maryland                 2157
Massachusetts            2451
Michigan                 3555
Minnesota                2017
Missouri                 2226
Montana                   398
Nebraska                  695
Nevada                   1037
New Hampshire             473
New 

In [168]:
cust_df["state"] = cust_df["state"].str.title()
cust_df["state"].replace('Mass', 'Massachusetts', inplace= True)

In [169]:
filter3 = cust_df[~cust_df["state"].isin(state_dict)]
print(filter3)
print(f'\n\t Number of customers not in USA: {len(filter3)}')

        customer_id         dob      state  start_balance creation_date
6921           7012  1992-02-17       -999       13340.92    2007-01-31
7877           7968  1997-05-04  Australia        9079.12    2007-01-31
37619         37710  1980-11-07  Australia        6341.96    2011-10-31
38804         38895  1969-03-25       -999        6235.20    2011-12-31
67699         67790  1998-06-12       -999        4862.36    2014-06-30
70608         70699  1987-04-07  Australia        4843.12    2014-10-31
82457         82548  1973-12-17        Unk       11648.96    2015-11-30
93005         93096  1975-04-30        Unk        6375.24    2017-05-31
104520       104611  1980-11-26        Unk        7073.92    2018-12-31

	 Number of customers not in USA: 9


In [170]:
l1 = list(filter3["state"])
for item in l1:
    cust_df["state"].replace(item, np.nan, inplace= True)

#### Checking dob

In [171]:
filter4 = cust_df[cust_df["dob"] <= '1940-01-01']
filter4.count()

customer_id      0
dob              0
state            0
start_balance    0
creation_date    0
dtype: int64

### looking at adding start balance to trans_df

In [172]:
in_dict = dict(zip(trans_df.customer_id, trans_df.account_id))

In [173]:
tempcust_df = cust_df.copy()

In [174]:
tempcust_df['account_id'] = tempcust_df['customer_id']
for key, value in in_dict.items():
    tempcust_df["account_id"].replace(key, value, inplace= True)

In [175]:
tempcust_df.head()

,customer_id,dob,state,start_balance,creation_date,account_id
0,91,1993-07-01,California,10180.56,2007-01-31,24137947
1,92,1985-12-05,New York,4757.68,2007-01-31,24137948
2,93,1987-11-19,Minnesota,6796.72,2007-01-31,24137949
3,94,1981-03-23,Minnesota,9870.48,2007-01-31,24137950
4,95,1970-04-06,California,12500.72,2007-01-31,24137951


In [176]:
#Transactions columns:
#['date', 'account_id', 'customer_id', 'amount', 'transaction_date', 'deposit', 'withdrawal', 'account_total', 'transaction_number']
tempcust_df["date"] = pd.to_datetime(tempcust_df["creation_date"])
tempcust_df["deposit"] = 0
tempcust_df["withdrawal"] = 0
tempcust_df["transaction_number"] = 0

tempcust_df.rename(columns= {'start_balance': 'amount', 'creation_date': 'transaction_date'}, inplace=True)
tempcust_df = tempcust_df.drop(['dob', 'state'], axis=1)


In [177]:
df3 = pd.concat([trans_df, tempcust_df], ignore_index= True)

In [178]:
trans_df = df3.sort_values(by=['customer_id', 'transaction_number'], ascending=True).reset_index(drop=True)

#### Adding an account_total column: 

By grouping by customer_id and sorting by transaction_number

In [179]:
trans_df['account_total'] = trans_df.sort_values(['customer_id', 'transaction_number'], ascending=True).groupby(['customer_id', 'account_id'])['amount'].cumsum()

In [197]:
display(trans_df)

,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal,transaction_number,account_total
0,2007-01-31,24137947,91,10180.56,2007-01-31,0.00,0.00,0,10180.56
1,2007-01-31,24137947,91,-5295.18,2007-01-16,0.00,-5295.18,1,4885.38
2,2007-01-31,24137947,91,3034.26,2007-01-31,3034.26,0.00,2,7919.64
3,2007-02-28,24137947,91,0.00,2007-02-28,0.00,0.00,3,7919.64
4,2007-03-31,24137947,91,-0.00,2007-03-11,0.00,-0.00,4,7919.64
...,...,...,...,...,...,...,...,...,...
5093983,2020-05-31,24253960,116104,2900.20,2020-05-31,2900.20,0.00,2,9909.26
5093984,2020-05-31,24253961,116105,8861.08,2020-05-31,0.00,0.00,0,8861.08
5093985,2020-05-31,24253961,116105,2246.93,2020-05-31,2246.93,0.00,1,11108.01
5093986,2020-05-31,24253962,116106,201.32,2020-05-31,0.00,0.00,0,201.32


## Testing Merge options

In [180]:
month_trans = trans_df.groupby(['customer_id', trans_df.date.dt.to_period("m")])["amount"].sum()

In [181]:
merge_df = pd.merge(trans_df[['account_id', 'customer_id', 'date', 'amount', 'account_total']], cust_df, on = 'customer_id', how = 'left')

In [182]:
display(merge_df.head())


,account_id,customer_id,date,amount,account_total,dob,state,start_balance,creation_date
0,24137947,91,2007-01-31,10180.56,10180.56,1993-07-01,California,10180.56,2007-01-31
1,24137947,91,2007-01-31,-5295.18,4885.38,1993-07-01,California,10180.56,2007-01-31
2,24137947,91,2007-01-31,3034.26,7919.64,1993-07-01,California,10180.56,2007-01-31
3,24137947,91,2007-02-28,0.00,7919.64,1993-07-01,California,10180.56,2007-01-31
4,24137947,91,2007-03-31,-0.00,7919.64,1993-07-01,California,10180.56,2007-01-31


In [183]:
sum_df = merge_df[['amount', 'start_balance']].sum(axis=1)

#### Creating temp table that is grouped by month

In [184]:
# temp_df = trans_df.groupby(['customer_id', 'account_id',
#                            trans_df.transaction_date.dt.to_period("M")], as_index = False).agg({'amount': sum, 'deposit': sum, 'withdrawal': sum})

# Use temp table below
temp_df = trans_df.groupby(['customer_id', 'account_id',
                            pd.Grouper(key = 'transaction_date', freq = "M")], 
                            as_index= False).agg({'amount': sum, 
                                                  'deposit': sum, 
                                                  'withdrawal': sum})

In [185]:
print(temp_df)

         customer_id  account_id transaction_date    amount  deposit  \
0                 91    24137947       2007-01-31   7919.64  3034.26   
1                 91    24137947       2007-02-28      0.00     0.00   
2                 91    24137947       2007-03-31      0.00     0.00   
3                 92    24137948       2007-01-31   4757.68     0.00   
4                 92    24137948       2007-02-28   1164.90  1164.90   
...              ...         ...              ...       ...      ...   
2129116       116102    24253958       2020-05-31   2760.01   872.99   
2129117       116103    24253959       2020-05-31  11060.70  3354.40   
2129118       116104    24253960       2020-05-31   9909.26  2900.20   
2129119       116105    24253961       2020-05-31  11108.01  2246.93   
2129120       116106    24253962       2020-05-31    258.31    56.99   

         withdrawal  
0          -5295.18  
1              0.00  
2              0.00  
3              0.00  
4              0.00  
...

In [186]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129121 entries, 0 to 2129120
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   customer_id       int64         
 1   account_id        int64         
 2   transaction_date  datetime64[ns]
 3   amount            float64       
 4   deposit           float64       
 5   withdrawal        float64       
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 97.5 MB


#### Merging data to look at each customer by month

In [187]:
tmer_df = pd.merge(cust_df, temp_df, on = 'customer_id', how = 'left')

In [188]:
tmer_df['transaction_total'] = tmer_df.sort_values(['customer_id', 'transaction_date'], ascending=True).groupby(['customer_id', 'account_id'])['amount'].cumsum()

In [189]:
tmer_df['account_total'] = tmer_df['transaction_total'] + tmer_df['start_balance']

In [190]:
tmer_df['account_total']

0          18100.20
1          18100.20
2          18100.20
3           9515.36
4          10680.26
             ...   
2129116     5870.97
2129117    23124.62
2129118    20920.62
2129119    19969.09
2129120      459.63
Name: account_total, Length: 2129121, dtype: float64

##### Creating a copy of the temp merge df to clean up a bit

In [191]:
monthly_look_df = tmer_df.copy()

In [192]:
monthly_look_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129121 entries, 0 to 2129120
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   customer_id        int64         
 1   dob                object        
 2   state              object        
 3   start_balance      float64       
 4   creation_date      datetime64[ns]
 5   account_id         int64         
 6   transaction_date   datetime64[ns]
 7   amount             float64       
 8   deposit            float64       
 9   withdrawal         float64       
 10  transaction_total  float64       
 11  account_total      float64       
dtypes: datetime64[ns](2), float64(6), int64(2), object(2)
memory usage: 194.9+ MB


##### Cleaning up this temp table 

so it just tells us about the account balance and transactions

In [193]:
monthly_look_df.drop(columns= ['dob', 'state', 'withdrawal', 'deposit'], inplace= True)
monthly_look_df["transaction_date"] = monthly_look_df.transaction_date.dt.to_period("M")
monthly_look_df = monthly_look_df.rename(columns={'transaction_date': "transaction_month", "amount": "transaction_amount"})

In [194]:
monthly_look_df.head()

,customer_id,start_balance,creation_date,account_id,transaction_month,transaction_amount,transaction_total,account_total
0,91,10180.56,2007-01-31,24137947,2007-01,7919.64,7919.64,18100.20
1,91,10180.56,2007-01-31,24137947,2007-02,0.00,7919.64,18100.20
2,91,10180.56,2007-01-31,24137947,2007-03,0.00,7919.64,18100.20
3,92,4757.68,2007-01-31,24137948,2007-01,4757.68,4757.68,9515.36
4,92,4757.68,2007-01-31,24137948,2007-02,1164.90,5922.58,10680.26
